In [8]:
from src.data_loader.data_loader import BooksDataset
from src.models.mm_model import MmModel
import pandas as pd
from src.train import Trainer
import torch

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = BooksDataset(data_dir="../data/books")

In [4]:
model = MmModel(n_users=dataset.n_users,n_items=dataset.n_items,adjacency_matrix=dataset.get_dataset("adjacency_matrix"),embed_size=64,n_layers=3)
model.to(device)

MmModel(
  (E0): Embedding(48752, 64)
)

In [6]:
trainer = Trainer(model=model,dataset=dataset)

In [7]:
trainer.train(epochs=2,batch_size=1024)

100%|██████████| 15/15 [00:04<00:00,  3.33it/s]


KeyboardInterrupt: 

In [5]:
device

device(type='cuda')

In [15]:
# create a tenor with 3 items
item = torch.tensor([0,2,3]).to(device)
user = torch.tensor([0,2,3]).to(device)
torch.mm(user,item.view(-1,1))





RuntimeError: self must be a matrix